In [2]:
import numpy as np
import scipy

print("NumPy version:", np.__version__)
print("SciPy version:", scipy.__version__)


NumPy version: 1.26.4
SciPy version: 1.7.3


In [15]:
import requests
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np

API_KEY = ' 0VAJ8VF5PWQP2IFB'

def get_option_chain_data(symbol: str, expiry_date: str, side: str) -> pd.DataFrame:
    url = f'https://www.alphavantage.co/query?function=HISTORICAL_OPTIONS&symbol=IBM&date=2017-11-15&apikey= 0VAJ8VF5PWQP2IFB'
    
  
    response = requests.get(url)
    if response.status_code != 200:
        print("Error fetching data:", response.status_code)
        return pd.DataFrame()  

 
    options_data = response.json()
    if 'data' not in options_data:
        print("No option data found.")
        return pd.DataFrame()

    records = []
    for option in options_data['data']:
        if option['expiration'] == expiry_date and option['type'] == side:
            strike_price = float(option['strike'])
            bid_ask_price = float(option['bid']) if side == "PE" else float(option['ask'])
            
            records.append({
                'symbol': symbol,
                'strike_price': strike_price,
                'side': side,
                'bid/ask': bid_ask_price
            })


    df = pd.DataFrame(records)
    return df

def calculate_margin_and_premium(data: pd.DataFrame) -> pd.DataFrame:
    lot_size = 75  

    def get_margin_requirement():
        return 1950 


    data['margin_required'] = data.apply(lambda row: get_margin_requirement(), axis=1)
    data['premium_earned'] = data['bid/ask'] * lot_size

    return data


symbol = 'IBM'
expiry_date = '2017-11-17'
side = 'put' 


options_data = get_option_chain_data(instrument_name, expiry_date, side)

if not options_data.empty:
    result = calculate_margin_and_premium(options_data)
    print(result)
else:
    print("No data available.")


   symbol  strike_price side  bid/ask  margin_required  premium_earned
0     IBM          75.0  put     0.03             1950            2.25
1     IBM          80.0  put     0.03             1950            2.25
2     IBM          85.0  put     0.03             1950            2.25
3     IBM          90.0  put     0.03             1950            2.25
4     IBM          95.0  put     0.01             1950            0.75
5     IBM         100.0  put     0.01             1950            0.75
6     IBM         105.0  put     0.01             1950            0.75
7     IBM         110.0  put     0.03             1950            2.25
8     IBM         115.0  put     0.03             1950            2.25
9     IBM         120.0  put     0.02             1950            1.50
10    IBM         125.0  put     0.01             1950            0.75
11    IBM         127.0  put     0.03             1950            2.25
12    IBM         128.0  put     0.03             1950            2.25
13    

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

data = {
    'symbol': ['IBM'] * 10,
    'strike_price': [75, 80, 85, 90, 95, 100, 105, 110, 115, 120],
    'side': ['put'] * 10,
    'bid/ask': [0.03, 0.03, 0.03, 0.03, 0.01, 0.01, 0.01, 0.03, 0.03, 0.02],
    'margin_required': [1950] * 10,
    'premium_earned': [2.25, 2.25, 2.25, 2.25, 0.75, 0.75, 0.75, 2.25, 2.25, 1.50],
    'implied_volatility': [4.20, 3.43, 3.06, 3.10, 4.05, 4.12, 3.95, 4.02, 3.99, 4.08],
    'volume': [0, 10, 0, 5, 20, 0, 3, 0, 7, 10]
}

df = pd.DataFrame(data)

features = ['strike_price', 'bid/ask', 'implied_volatility', 'volume']
target = 'premium_earned'

X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)

predictions = pd.DataFrame({
    'Actual Premium': y_test,
    'Predicted Premium': y_pred
})
print(predictions)


Mean Squared Error: 6.990096481163386e-27
Mean Absolute Error: 8.348877145181177e-14
   Actual Premium  Predicted Premium
8            2.25               2.25
1            2.25               2.25


In [12]:
import pandas as pd


sample_data = {
    'data': [
        {'symbol': 'IBM', 'strike': 150.0, 'type': 'put', 'bid': 3.05, 'ask': 2.95},
        {'symbol': 'IBM', 'strike': 155.0, 'type': 'put', 'bid': 8.10, 'ask': 7.90},
        {'symbol': 'IBM', 'strike': 160.0, 'type': 'put', 'bid': 12.95, 'ask': 12.75},
        {'symbol': 'IBM', 'strike': 150.0, 'type': 'call', 'bid': 3.10, 'ask': 3.00},
        {'symbol': 'IBM', 'strike': 155.0, 'type': 'call', 'bid': 7.90, 'ask': 7.70}
    ]
}

def get_option_chain_data(symbol: str, expiry_date: str, side: str, sample_data=None) -> pd.DataFrame:
  
    if side not in ["PE", "CE"]:
        raise ValueError("Invalid side input. Use 'PE' for put options or 'CE' for call options.")

   
    if sample_data is not None:
        print("Using sample data for testing.")
        data = sample_data
    else:
        print("No sample data provided; normally, this would call the API.")
        return pd.DataFrame()  

    if 'data' not in data:
        print("Error: No data field in sample data.")
        return pd.DataFrame()

    df = pd.DataFrame(data['data'])
    option_type = 'put' if side == 'PE' else 'call'
    df_filtered = df[df['type'] == option_type].copy()

    if df_filtered.empty:
        print("Warning: No matching options found in sample data.")
        return pd.DataFrame()

    df_filtered['bid/ask'] = df_filtered['bid'] if side == 'PE' else df_filtered['ask']

    return df_filtered[['symbol', 'strike', 'type', 'bid/ask']]

test_data = get_option_chain_data('IBM', '2024-11-10', 'PE', sample_data=sample_data)
print("Test data output:\n", test_data)


Using sample data for testing.
Test data output:
   symbol  strike type  bid/ask
0    IBM   150.0  put     3.05
1    IBM   155.0  put     8.10
2    IBM   160.0  put    12.95


In [17]:
import requests

def fetch_options_data(symbol, expiration, strike_price):
    try:
        validate_data({
            'symbol': symbol,
            'expiration': expiration,
            'strike_price': strike_price
        })

        response = requests.get("https://sandbox-api.com/options", params={
            "symbol": symbol,
            "expiration": expiration,
            "strike_price": strike_price
        })

        response.raise_for_status()
        
        data = response.json()
        if data.get("message") != "success" or not data.get("data"):
            raise ValueError("No data available in API response")

        return data

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"Request error: {req_err}")
    except ValueError as val_err:
        print(f"Data error: {val_err}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
